<a href="https://colab.research.google.com/github/karabhilash/CS6910-Fun_of_DL/blob/main/Assignment3/Assignment3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


!pip install wandb

!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar

!tar -xf dakshina_dataset_v1.0.tar



In [ ]:
!ls 

dakshina_dataset_v1.0  dakshina_dataset_v1.0.tar  sample_data


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Activation
from keras.layers import Concatenate,TimeDistributed
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU,SimpleRNN

import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense,BatchNormalization
from keras.layers import Flatten
import math
from keras.layers import Dropout
import random
import os

In [ ]:
train_file="dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
val_file="dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"
test_file="dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"


In [ ]:
train_data = pd.read_csv(train_file,sep='\t',names=['target','src','occurances'])
val_data = pd.read_csv(val_file,sep='\t',names=['target','src','occurances'])
test_data = pd.read_csv(test_file,sep='\t',names=['target','src','occurances'])



In [ ]:
def character_set(source,target):
    s_char,t_char= set(),set()
    for s in source:
        for ch in s:
            s_char.add(ch)
    for s in target:
        for ch in s:
            t_char.add(ch)
    s_char.add(" ")
    t_char.add(" ")
    return sorted(list(s_char)),sorted(list(t_char))

def create_lookup(tokens):
    stoi = dict(zip(tokens,range(len(tokens))))
    itos = {v: k for k, v in stoi.items()}
    return stoi,itos

def encode(source,target,source_lex,target_lex,source_stoi,target_stoi):

    num_source_tokens = len(source_lex)
    num_target_tokens = len(target_lex)
    max_source_length = max([len(s) for s in source])
    max_target_length = max([len(s) for s in target])
    num_samples = len(source)

    encoder_input= np.zeros((num_samples,max_source_length,num_source_tokens))
    decoder_input=np.zeros((num_samples,max_target_length,num_target_tokens))
    decoder_target=np.zeros((num_samples,max_target_length,num_target_tokens))

    for i in range(num_samples):
        for j,char in enumerate(source[i]):
            encoder_input[i,j,source_stoi[char]] = 1.0
        encoder_input[i,j+1:,source_stoi[" "]] = 1.0
        for j,char in enumerate(target[i]):
            decoder_input[i,j,target_stoi[char]] = 1.0
        decoder_input[i,j+1:,target_stoi[" "]] = 1.0
        for j,char in enumerate(target[i][1:]):
            decoder_target[i,j-1,target_stoi[char]] = 1.0
        decoder_target[i,j:,target_stoi[" "]] = 1.0
    return encoder_input,decoder_input,decoder_target

    



In [ ]:
source = [str(s) for s in list(train_data["src"])]
target = [str(s) for s in list(train_data["target"])]
target = ["\t"+s+"\n" for s in target]
source_tokens,target_tokens = character_set(source,target)

source_stoi,source_itos = create_lookup(source_tokens)
target_stoi,target_itos = create_lookup(target_tokens)

train_encoder_input,train_decoder_input,train_decoder_target = encode(source,target,source_tokens,target_tokens,source_stoi,target_stoi)

In [ ]:

max_source_length = max([len(s) for s in source])
max_target_length = max([len(s) for s in target])

In [ ]:
#preprocess val data
source_val = [str(s) for s in list(val_data["src"])]
target_val = [str(s) for s in list(val_data["target"])]
target_val = ["\t"+s+"\n" for s in target_val]
val_encoder_input,val_decoder_input,val_decoder_target = encode(source_val,target_val,source_tokens,target_tokens,source_stoi,target_stoi)

In [ ]:
#preprocess test data
source_test = [str(s) for s in list(test_data["src"])]
target_test = [str(s) for s in list(test_data["target"])]
target_test = ["\t"+s+"\n" for s in target_test]
test_encoder_input,test_decoder_input,test_decoder_target = encode(source_test,target_test,source_tokens,target_tokens,source_stoi,target_stoi)

In [ ]:
def build_model_singleLink(cell_type,Dimensions,num_encoders,num_decoders,num_dense_neurons,dropout,extra_dense_layer,source_stoi,target_stoi):
    num_encoder_tokens = len(source_stoi)
    num_decoder_tokens = len(target_stoi)
    
    if cell_type=='RNN':
        encoder_inputs = Input(shape=(None,num_encoder_tokens))
        encoder_i = encoder_inputs
        encoder_states=[]
        for i in range(num_encoders):
            encoder_i,state = SimpleRNN(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "enc_"+str(i))(encoder_i)
            encoder_states += [state]
        encoder_output  = encoder_i

        decoder_inputs = Input(shape=(None,num_decoder_tokens))
        decoder_i = decoder_inputs

        for i in range(num_decoders):
            decoder_i,state = SimpleRNN(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "dec_"+str(i))(decoder_i,initial_state=encoder_states[-1])
        
        decoder_output = decoder_i
        if extra_dense_layer == True:
            decoder_output = Dense(num_dense_neurons,activation='relu',name='dense')(decoder_output)
        decoder_output = Dense(num_decoder_tokens,activation='softmax', name = 'final')(decoder_output)
        model = Model([encoder_inputs, decoder_inputs], decoder_output)

    if cell_type=='GRU':
        encoder_inputs = Input(shape=(None,num_encoder_tokens))
        encoder_i = encoder_inputs
        encoder_states=[]
        for i in range(num_encoders):
            encoder_i,state = GRU(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "enc_"+str(i))(encoder_i)
            encoder_states += [state]
        encoder_output  = encoder_i

        decoder_inputs = Input(shape=(None,num_decoder_tokens))
        decoder_i = decoder_inputs

        for i in range(num_decoders):
            decoder_i,state = GRU(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "dec_"+str(i))(decoder_i,initial_state=encoder_states[-1])
        
        decoder_output = decoder_i
        if extra_dense_layer == True:
            decoder_output = Dense(num_dense_neurons,activation='relu',name='dense')(decoder_output)
        decoder_output = Dense(num_decoder_tokens,activation='softmax', name = 'final')(decoder_output)
        model = Model([encoder_inputs, decoder_inputs], decoder_output)
        

    if cell_type=='LSTM':
        encoder_inputs = Input(shape=(None,num_encoder_tokens))
        encoder_i = encoder_inputs
        encoder_states=[]
        for i in range(num_encoders):
            encoder_i,s1,s2 = LSTM(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "enc_"+str(i))(encoder_i)
            encoder_states += [[s1,s2]]
        encoder_output  = encoder_i

        decoder_inputs = Input(shape=(None,num_decoder_tokens))
        decoder_i = decoder_inputs

        for i in range(num_decoders):
            decoder_i,s1,s2 = LSTM(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "dec_"+str(i))(decoder_i,initial_state=encoder_states[-1])
        
        decoder_output = decoder_i
        if extra_dense_layer == True:
            decoder_output = Dense(num_dense_neurons,activation='relu',name='dense')(decoder_output)
        decoder_output = Dense(num_decoder_tokens,activation='softmax', name = 'final')(decoder_output)
        model = Model([encoder_inputs, decoder_inputs], decoder_output)


    return model

        
def build_model(cell_type,Dimensions,num_encoders,num_decoders,num_dense_neurons,dropout,extra_dense_layer,source_stoi,target_stoi):
    num_encoder_tokens = len(source_stoi)
    num_decoder_tokens = len(target_stoi)
    
    if cell_type=='RNN':
        encoder_inputs = Input(shape=(None,num_encoder_tokens))
        encoder_i = encoder_inputs
        encoder_states=[]
        for i in range(num_encoders):
            encoder_i,state = SimpleRNN(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "enc_"+str(i))(encoder_i)
            encoder_states += [state]
        encoder_output  = encoder_i

        decoder_inputs = Input(shape=(None,num_decoder_tokens))
        decoder_i = decoder_inputs

        for i in range(num_decoders):
            decoder_i,state = SimpleRNN(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "dec_"+str(i))(decoder_i,initial_state=encoder_states[i])
        
        decoder_output = decoder_i
        if extra_dense_layer == True:
            decoder_output = Dense(num_dense_neurons,activation='relu',name='dense')(decoder_output)
        decoder_output = Dense(num_decoder_tokens,activation='softmax', name = 'final')(decoder_output)
        model = Model([encoder_inputs, decoder_inputs], decoder_output)

    if cell_type=='GRU':
        encoder_inputs = Input(shape=(None,num_encoder_tokens))
        encoder_i = encoder_inputs
        encoder_states=[]
        for i in range(num_encoders):
            encoder_i,state = GRU(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "enc_"+str(i))(encoder_i)
            encoder_states += [state]
        encoder_output  = encoder_i

        decoder_inputs = Input(shape=(None,num_decoder_tokens))
        decoder_i = decoder_inputs

        for i in range(num_decoders):
            decoder_i,state = GRU(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "dec_"+str(i))(decoder_i,initial_state=encoder_states[i])
        
        decoder_output = decoder_i
        if extra_dense_layer == True:
            decoder_output = Dense(num_dense_neurons,activation='relu',name='dense')(decoder_output)
        decoder_output = Dense(num_decoder_tokens,activation='softmax', name = 'final')(decoder_output)
        model = Model([encoder_inputs, decoder_inputs], decoder_output)
        

    if cell_type=='LSTM':
        encoder_inputs = Input(shape=(None,num_encoder_tokens))
        encoder_i = encoder_inputs
        encoder_states=[]
        for i in range(num_encoders):
            encoder_i,s1,s2 = LSTM(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "enc_"+str(i))(encoder_i)
            encoder_states += [[s1,s2]]
        encoder_output  = encoder_i

        decoder_inputs = Input(shape=(None,num_decoder_tokens))
        decoder_i = decoder_inputs

        for i in range(num_decoders):
            decoder_i,s1,s2 = LSTM(Dimensions,return_state=True,return_sequences=True,dropout=dropout,name= "dec_"+str(i))(decoder_i,initial_state=encoder_states[i])
        
        decoder_output = decoder_i
        if extra_dense_layer == True:
            decoder_output = Dense(num_dense_neurons,activation='relu',name='dense')(decoder_output)
        decoder_output = Dense(num_decoder_tokens,activation='softmax', name = 'final')(decoder_output)
        model = Model([encoder_inputs, decoder_inputs], decoder_output)


    return model

        

In [ ]:
def train_simple():
    model = build_model("RNN",256,3,3,64,0.3,False,source_stoi,target_stoi)
    model.summary()
    model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=["accuracy"])
    model.fit([train_encoder_input,train_decoder_input],train_decoder_target,validation_data=([val_encoder_input,val_decoder_input],val_decoder_target) ,epochs = 3)


In [ ]:
train_simple()

In [ ]:
best_model = None
best_val_acc = 0.0
best_CONFIG = None
import wandb
from wandb.keras import WandbCallback
def paramTuning():
    global best_model,best_val_acc,best_CONFIG
    config = dict(
        cell = "LSTM",
        num_dense_neurons = 128,
        dimensions= 256,
        num_encoder_layers = 1,
        num_decoder_layers = 1,
        dropout = 0.3,
        epochs = 2,
        extra_dense_layer = False,
        build_model_type = False
    )

    wandb.init(config=config)
    CONFIG = wandb.config
    if CONFIG.build_model_type:
        model = build_model(CONFIG.cell,CONFIG.dimensions,CONFIG.num_encoder_layers,CONFIG.num_encoder_layers,CONFIG.num_dense_neurons,CONFIG.dropout,CONFIG.extra_dense_layer,source_stoi,target_stoi)
    else:
        model = build_model_singleLink(CONFIG.cell,CONFIG.dimensions,CONFIG.num_encoder_layers,CONFIG.num_decoder_layers,CONFIG.num_dense_neurons,CONFIG.dropout,CONFIG.extra_dense_layer,source_stoi,target_stoi)

    model.summary()
    model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=["accuracy"])
    h = model.fit([train_encoder_input,train_decoder_input],train_decoder_target,validation_data=([val_encoder_input,val_decoder_input],val_decoder_target),epochs = CONFIG.epochs ,callbacks=[WandbCallback()])
    print(h.history['val_accuracy'][-1])
    if h.history['val_accuracy'][-1] > best_val_acc:
        best_model = model
        best_val_acc=h.history['val_accuracy'][-1]
        best_CONFIG = CONFIG

In [ ]:
sweep_config = {
    "name": "non-attention",
    "method": "bayes",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        
        "cell": 
            {"values": ["RNN", "GRU", "LSTM"]},
        
        "dimensions": 
            {"values": [256, 128, 64]},
        
        "extra_dense_layer": 
            {"values": [True,False]},
        
        "optimiser": 
            {"values": ["rmsprop", "adam"]},
        
        "num_encoder_layers": 
            {"values": [1, 2, 3]},
        
        "num_decoder_layers": 
            {"values": [1, 2, 3]},
        
        "dropout": {"values": [0.1, 0.2, 0.3]},
        
        "epochs": {"values": [5,10]},
        
        "batch_size": {"values": [32, 64]},
        "build_model_type":
            {'values':[True,False]} # if True,ith decoder layers initial state corresponds to ith encoder layer output state
    },
}

sweep_id = wandb.sweep(sweep_config, project="CS6910-Assignment-3-1",entity='abhilash-kar0')




<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 5ycfhsjc
Sweep URL: https://wandb.ai/abhilash-kar0/CS6910-Assignment-3-1/sweeps/5ycfhsjc


In [ ]:
a='7206be0b2a6d421b3fa0fe354acc489584df6053'

In [ ]:
wandb.agent(sweep_id, paramTuning, count = 3)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: as6muu0y with config:
wandb: 	batch_size: 32
wandb: 	build_model_type: False
wandb: 	cell: RNN
wandb: 	dimensions: 64
wandb: 	dropout: 0.1
wandb: 	epochs: 5
wandb: 	extra_dense_layer: True
wandb: 	num_decoder_layers: 2
wandb: 	num_encoder_layers: 1
wandb: 	optimiser: adam


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 27)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 66)]   0                                            
__________________________________________________________________________________________________
enc_0 (SimpleRNN)               [(None, None, 64), ( 5888        input_1[0][0]                    
__________________________________________________________________________________________________
dec_0 (SimpleRNN)               [(None, None, 64), ( 8384        input_2[0][0]                    
                                                                 enc_0[0][1]                  

epoch,4
loss,121.56987
accuracy,0.71035
val_loss,199.79053
val_accuracy,0.64364
_runtime,543
_timestamp,1624991367
_step,4
best_val_loss,45.62489
best_epoch,0


epoch,▁▃▅▆█
loss,▁▃▅▅█
accuracy,█▆▁▂▂
val_loss,▁▃▄▆█
val_accuracy,█▆▄▇▁
_runtime,▁▃▅▆█
_timestamp,▁▃▅▆█
_step,▁▃▅▆█


wandb: ERROR Run as6muu0y errored: TypeError("'>' not supported between instances of 'float' and 'list'")
wandb: Agent Starting Run: bu27udhu with config:
wandb: 	batch_size: 32
wandb: 	build_model_type: True
wandb: 	cell: GRU
wandb: 	dimensions: 256
wandb: 	dropout: 0.1
wandb: 	epochs: 5
wandb: 	extra_dense_layer: True
wandb: 	num_decoder_layers: 1
wandb: 	num_encoder_layers: 2
wandb: 	optimiser: adam


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 27)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 66)]   0                                            
__________________________________________________________________________________________________
enc_0 (GRU)                     [(None, None, 256),  218880      input_1[0][0]                    
__________________________________________________________________________________________________
dec_0 (GRU)                     [(None, None, 256),  248832      input_2[0][0]                    
                                                                 enc_0[0][1]                  

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
num_dec_layers=best_CONFIG.num_decoder_layers
num_enc_layers=best_CONFIG.num_encoder_layers
if best_CONFIG.build_model_type:
    num_dec_layers=best_CONFIG.num_encoder_layers
dim = best_CONFIG.dimensions
model = best_model
encoder_inputs = model.input[0]  # input_1

encoder_states = []
for i in range(num_enc_layers):
    en_res = model.get_layer('enc_'+str(i)).output  
    encoder_outputs = en_res[0] 
    encoder_states += [en_res[1:]]
    
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_i = decoder_inputs
decoder_states_inputs = []
decoder_states = []

for i in range(num_dec_layers):
    if best_CONFIG.cell == 'LSTM':
        decoder_state_input_h = Input(shape=(dim,))
        decoder_state_input_c = Input(shape=(dim,))
        decoder_states_inputs += [decoder_state_input_h, decoder_state_input_c]
    elif best_CONFIG.cell == 'GRU':
        decoder_states_inputs += [Input(shape=(dim,))]
    elif best_CONFIG.cell == 'RNN':
        decoder_states_inputs += [Input(shape=(dim,))]
    
    
    de_out = model.get_layer('dec_'+str(i))(decoder_i, initial_state=decoder_states_inputs[-1])
    decoder_i = de_out[0]
    decoder_states += [de_out[1:]]
    
decoder_outputs = decoder_i
if best_CONFIG.extra_dense_layer:
    decoder_outputs = model.get_layer('dense')(decoder_outputs)
decoder_dense = model.get_layer('dense_1')
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [ ]:


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(target_tokens)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_stoi["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        outputs = decoder_model.predict([target_seq] + states_value)
        output_tokens, states_value = outputs[0], outputs[1:]

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_itos[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_target_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, len(target_tokens)))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
#         states_value = [h, c]

    return decoded_sentence



In [ ]:
encoder_model.summary(),decoder_model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 27)]        0         
_________________________________________________________________
enc_0 (GRU)                  [(None, None, 256), (None 218880    
_________________________________________________________________
enc_1 (GRU)                  [(None, None, 256), (None 394752    
Total params: 613,632
Trainable params: 613,632
Non-trainable params: 0
_________________________________________________________________
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 66)]   0                                            
___________________________________________________________________

(None, None)